In [27]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import requests
# import pandas as pd
import re

In [46]:
######## Get genre names

url = 'https://www.imdb.com/feature/genre'
# url1 = 'https://finance.yahoo.com/quote/TATAPOWER.NS/balance-sheet?p=TATAPOWER.NS'
headers = {'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Mobile Safari/537.36'}
response = get(url, headers=headers)
doc = BeautifulSoup(response.text, 'html.parser')
genre_links = doc.find_all('a' , class_ = "ipc-chip ipc-chip--on-base-accent2")
genre_links = str(genre_links)
doc_genre_links = BeautifulSoup(genre_links, 'html.parser')
genre_list = doc_genre_links.find_all(class_ = "ipc-chip__text")
genre_list

genre_names = [genre.text for genre in genre_list]

genre_unique_names = []
seen_names = set()

for name in genre_names:
    if name not in seen_names:
        genre_unique_names.append(name)
        seen_names.add(name)
        

In [47]:
####### Genrate links of individual genres

urlMain = "https://www.imdb.com"
genre_links = []

for genre in genre_unique_names:
    urlGenre = urlMain + "/search/title/?genres=" + genre.lower()
    genre_links.append(urlGenre)
    
    
genre_links = genre_links[0:20]
genre_unique_names = genre_unique_names[0:20]

In [36]:
##### Generate movie links for a particular genre

def getGenrePage(link):
    genre_res_mov = get(link, headers=headers)
    genre_mov_doc = BeautifulSoup(genre_res_mov.text, 'html.parser')
    genre_movs = genre_mov_doc.find_all(class_ = "ipc-title-link-wrapper")
    genre_movs = genre_movs[:20]
    return genre_movs

    

In [50]:

########## Function of finding director names of movies of a particular genre

def getDirNames(genre_movs,genre):

    df_genre = pd.DataFrame(columns=['Directors'])

    for mov_link in genre_movs:
        movie_href = mov_link.get('href')
        url_mov = "https://www.imdb.com" + movie_href
        movie_data = get(url_mov, headers=headers)
        doc_mov = BeautifulSoup(movie_data.text, 'html.parser')
        list_items = doc_mov.find('li', class_='ipc-metadata-list__item')
        list_a = list_items.find_all(class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
        html_string = str(list_a)
        dir_names_parsed = BeautifulSoup(html_string, 'html.parser')
        dir_names = dir_names_parsed.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
        director_names = [director.text for director in dir_names]
        director_unique_names = list(dict.fromkeys(director_names))
        df_genre = df_genre.append({'Directors': ', '.join(director_unique_names)}, ignore_index=True)

    movie_names = df_pivoted[genre]
    df_genre['Movie Name'] = movie_names
    df_genre.index = movie_names
    df_genre = df_genre[['Movie Name' , 'Directors']]
    return df_genre

In [51]:
####### Code for extracting yor, runtime and content rating

def movieProps(genre_movs,movie_names,df_test):

    for mov_link,movie in zip(genre_movs,movie_names):
        movie_href = mov_link.get('href')
        url_mov = "https://www.imdb.com" + movie_href
        movie_data = get(url_mov, headers=headers)
        doc_mov = BeautifulSoup(movie_data.text, 'html.parser')
        doc_mov_div = doc_mov.find_all('div' , class_='sc-e226b0e3-3 dwkouE' )
        doc_mov_div = str(doc_mov_div);
        doc_mov_div = BeautifulSoup(doc_mov_div, 'html.parser')
        yearList = doc_mov_div.find_all('li', class_='ipc-inline-list__item')
        html_string = str(yearList)
        data_parsed = BeautifulSoup(html_string, 'html.parser')
        movie_props = data_parsed.find_all(class_='ipc-link ipc-link--baseAlt ipc-link--inherit-color')

        props = [prop.text for prop in movie_props]
        prop_names = []
        seen_names = set()

        for name in props:
            if name not in seen_names:
                prop_names.append(name)
                seen_names.add(name)

        prop_names.append(yearList[-1].text)
        
        doc_rating_div = doc_mov.find_all('div' , class_='sc-3a4309f8-1 dggvUg')
        doc_rating_div = str(doc_rating_div)
        doc_rating_div = BeautifulSoup(doc_rating_div, 'html.parser')
        doc_mov_rating = doc_mov.find_all('span' , class_='sc-bde20123-1 cMEQkK')
        df_test.loc[movie, 'Rating'] = doc_mov_rating[0].text

        df_test.loc[movie, 'Year of Release'] = prop_names[0]
        df_test.loc[movie, 'Content Rating'] = prop_names[1]
        df_test.loc[movie, 'Runtime'] = prop_names[2]

    
    return df_test


In [52]:
##### Dataframe maker from gpt (DF_PIVOTED)
##### Needed for getting movie_names

import pandas as pd

urlMain = "https://www.imdb.com"
headers = {'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Mobile Safari/537.36'}

df = pd.DataFrame(columns=['Genre', 'Title'])


for genre in genre_unique_names:
    urlGenre = urlMain + "/search/title/?genres=" + genre.lower()
    genre_res = get(urlGenre, headers=headers)
    doc_genre = BeautifulSoup(genre_res.text, 'html.parser')
    
    genre_movie_names = [re.sub(r'^\d+\.\s*', '', name.text) for name in doc_genre.find_all(class_="ipc-title__text")[1:21]]
    
    df = df.append(pd.DataFrame({'Genre': [genre] * len(genre_movie_names), 'Title': genre_movie_names}))
    
df_pivoted = df.pivot(index=None, columns='Genre', values='Title')
df_pivoted = df_pivoted.apply(lambda x: pd.Series(x.dropna().values)).fillna('')



In [65]:

def reviewGenerator(userReviewLinks,movie_names,DATA):
    
    for review_link,movie in zip(userReviewLinks,movie_names):
        review_data = get(review_link, headers=headers)
        doc_review = BeautifulSoup(review_data.text, 'html.parser')
        doc_review_div = doc_review.find_all('div' , class_='lister-list')
        doc_review_div = BeautifulSoup(str(doc_review_div), 'html.parser')
        allReviewContent = doc_review_div.find_all('div' , class_ = 'text')
        allReviewDate = doc_review_div.find_all('span' , class_ = 'review-date')
        allReviewRating = doc_review_div.find_all('span' , class_ = 'rating-other-user-rating')
        doc_rating_all = BeautifulSoup(str(allReviewRating), 'html.parser')
        finalRatings = doc_rating_all.find_all('span', class_ = '')
        finalRatings = finalRatings[:10]
        allReviewDate = allReviewDate[:10]
        allReviewContent = allReviewContent[:10]
        finalRatingList = []
        finalContentList = []
        finalDateList = []
        for rating,content,date in zip(finalRatings,allReviewContent,allReviewDate):
            finalRatingList.append(rating.text)
            finalContentList.append(content.text)
            finalDateList.append(date.text)

        listr = []
        for rating,content,date in zip(finalRatings,allReviewContent,allReviewDate):
            listr.append({'text':content.text, 'review-date' : date.text , 'review-score' : rating.text })

        DATA['Action'][movie]['Reviews'] = listr
    
    return DATA
    

In [66]:
#### Generate url for user reviews

def generateReviewLinks(genre_movs,movie_names):
    url_imdb = "https://www.imdb.com"

    userReviewLinks = []

    for mov_link,movie in zip(genre_movs,movie_names):
        movie_href = mov_link.get('href')
        url_mov = "https://www.imdb.com" + movie_href
        movie_data = get(url_mov, headers=headers)
        doc_mov = BeautifulSoup(movie_data.text, 'html.parser')
        doc_review_div = doc_mov.find_all('div' , class_='sc-3a4309f8-1 dggvUg')
        doc_review_div = str(doc_review_div)
        doc_review_div = BeautifulSoup(doc_review_div, 'html.parser')
        doc_review = doc_mov.find_all('li' , class_='ipc-inline-list__item')
        doc_review = doc_review[1]
        doc_review = str(doc_review)
        doc_review_parsed = BeautifulSoup(doc_review, 'html.parser')
        reviews_link = doc_review_parsed.find_all('a' , class_='ipc-link ipc-link--baseAlt ipc-link--inherit-color')
        reviews_link = reviews_link[0]
        reviews_link = reviews_link.get('href')
        reviews_url = url_imdb + reviews_link
        print(reviews_url)
        userReviewLinks.append(reviews_url)
    
    return userReviewLinks

In [67]:
#### Generate url for user reviews

def userReviewUrlGenrator(genre_movs,movie_names):
    url_imdb = "https://www.imdb.com"

    userReviewLinks = []

    for mov_link,movie in zip(genre_movs,movie_names):
        movie_href = mov_link.get('href')
        url_mov = "https://www.imdb.com" + movie_href
        movie_data = get(url_mov, headers=headers)
        doc_mov = BeautifulSoup(movie_data.text, 'html.parser')
        doc_review_div = doc_mov.find_all('div' , class_='sc-3a4309f8-1 dggvUg')
        doc_review_div = str(doc_rating_div)
        doc_review_div = BeautifulSoup(doc_review_div, 'html.parser')
        doc_review = doc_mov.find_all('li' , class_='ipc-inline-list__item')
        doc_review = doc_review[1]
        doc_review = str(doc_review)
        doc_review_parsed = BeautifulSoup(doc_review, 'html.parser')
        reviews_link = doc_review_parsed.find_all('a' , class_='ipc-link ipc-link--baseAlt ipc-link--inherit-color')
        reviews_link = reviews_link[0]
        reviews_link = reviews_link.get('href')
        reviews_url = url_imdb + reviews_link
        print(reviews_url)
        userReviewLinks.append(reviews_url)
    
    return userReviewLinks

In [68]:
#### Testing JSON generation

import json

DATA = {}
DATA['Action'] = {}
movie_data = get(genre_links[0], headers=headers)
doc_mov = BeautifulSoup(movie_data.text, 'html.parser')
genre_movs = getGenrePage(genre_links[0])
df_directors = getDirNames(genre_movs,'Action')
df_directors = movieProps(genre_movs,movie_names,df_directors)
movie_names = df_pivoted['Action']
for movie in movie_names:
    DATA['Action'][movie] = {
        'Directors':df_directors.loc[movie , 'Directors'],
        'Year of Release': df_directors.loc[movie , 'Year of Release'],
        'Content Rating':df_directors.loc[movie , 'Content Rating'],
        'Runtime':df_directors.loc[movie , 'Runtime'],
        'IMDB Rating' : df_directors.loc[movie , 'Rating']
    }

with open('data.json', 'w') as fp:
    json.dump(DATA, fp)

In [ ]:
# Display the dictionary
DATA

In [ ]:
#### FINAL JSON GENERATION (Still in progress)
#### This cell provides the dictionary with all the required data

DATA = {}

for link,genre in zip(genre_links,genre_unique_names):
    movie_data = get(link, headers=headers)
    doc_mov = BeautifulSoup(movie_data.text, 'html.parser')
    genre_movs = getGenrePage(link)
    df_directors = getDirNames(genre_movs,genre)
    movie_names = df_pivoted[genre]
    userReviewLinks = generateReviewLinks(genre_movs,movie_names)
    df_directors = movieProps(genre_movs,movie_names,df_directors)
    DATA[genre] = {}
    for movie in movie_names:
        DATA[genre][movie] = {
            'Directors':df_directors.loc[movie , 'Directors'],
            'Year of Release': df_directors.loc[movie , 'Year of Release'],
            'Content Rating':df_directors.loc[movie , 'Content Rating'],
            'Runtime':df_directors.loc[movie , 'Runtime'],
            'IMDB Rating' : df_directors.loc[movie , 'Rating']
        }
        
    
    DATA = reviewGenerator(userReviewLinks,movie_names , DATA)
    del df_directors